<a href="https://colab.research.google.com/github/smazzilli/LO/blob/main/Sparrow_Nutrient_Loads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %% [code]
#SM comment edit May 9th, 2025. edit 2

## Initialization (once per session)
# Installs
!pip install xlsxwriter openpyxl

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports & constants
import pandas as pd, numpy as np, calendar
from google.colab import files
SHARED = '/content/drive/MyDrive/SPARROW Google CoLab Files'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# PART 1: Aggregate Predicted Nutrient Loads by Source & WRIA (predict_puget_tn.csv → annual_quarterly_loads.xlsx)
# ──────────────────────────────────────────────────────────────────────────────

# 1) Read in your Puget TN predictions CSV
df = pd.read_csv(f'{SHARED}/predict_puget_tn.csv', dtype={8: str})

# 2) Preprocessing: combine WRIA 3 & 4, rename quarters, map WRIA numbers to names
df['WRIA'] = df['WRIA'].apply(lambda x: '3&4' if x in [3, 4] else x)
df['quarter'] = df['quarter'].map({1:'Winter', 2:'Spring', 3:'Summer', 4:'Fall'})
wria_mapping = {
    1: 'Nooksack', 2: 'San Juan', '3&4': 'Skagit - Samish', 5: 'Stillaguamish',
    6: 'Island', 7: 'Snohomish', 8: 'Cedar- Sammamish', 9: 'Duwamish - Green',
    10: 'Puyallup - White', 11: 'Nisqually', 12: 'Chambers - Clover',
    13: 'Deschutes', 14: 'Kennedy - Goldsborough', 15: 'Kitsap',
    16: 'Skokomish - Dosewallips', 17: 'Quilcene - Snow',
    18: 'Elwha - Dungeness', 19: 'Lyre - Hoko'
}
df['WRIA'] = df['WRIA'].map(wria_mapping)

# 3) Define and sanitize your source columns
sources = [
    'PLOAD_INC_FOREIGN','PLOAD_INC_POINT1_KG','PLOAD_INC_CAFOS','PLOAD_INC_SEPTIC',
    'PLOAD_INC_AG_KG','PLOAD_INC_ATM_KG','PLOAD_INC_URB_KM2','PLOAD_INC_NFIX_M2',
    'PLOAD_INC_STORAGE','PLOAD_INC_TOTAL_ND','PLOAD_INC_TOTAL'
]
for col in sources:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 4) Compute loads in Gg (kg ÷ 1e6)
annual_load = (df
    .groupby(['WRIA','year'])[sources]
    .sum()
    .reset_index()
)
annual_load[sources] /= 1e6
annual_load['aquatic decay'] = (
    annual_load['PLOAD_INC_TOTAL_ND'] - annual_load['PLOAD_INC_TOTAL']
)

annual_mean_load = (annual_load
    .groupby('WRIA')[sources]
    .mean()
    .reset_index()
)
annual_mean_load[sources] /= 1  # already in Gg; keep same
annual_mean_load['aquatic decay'] = (
    annual_mean_load['PLOAD_INC_TOTAL_ND'] - annual_mean_load['PLOAD_INC_TOTAL']
)

quarterly_by_year = (df
    .groupby(['WRIA','quarter','year'])[sources]
    .sum()
    .reset_index()
)
quarterly_by_year[sources] /= 1e6
quarterly_by_year['aquatic decay'] = (
    quarterly_by_year['PLOAD_INC_TOTAL_ND'] - quarterly_by_year['PLOAD_INC_TOTAL']
)

quarterly_mean_load = (quarterly_by_year
    .groupby(['WRIA','quarter'])[sources + ['aquatic decay']]
    .mean()
    .reset_index()
)

# 5) Rename for presentation
rename_sources = {
    'PLOAD_INC_TOTAL': 'Total',
    'PLOAD_INC_FOREIGN': 'Inflow from Canada',
    'PLOAD_INC_POINT1_KG': 'Permitted Treated Wastewater',
    'PLOAD_INC_CAFOS': 'Animal Feeding Operations',
    'PLOAD_INC_SEPTIC': 'On-site Treated Wastewater',
    'PLOAD_INC_AG_KG': 'Fertilizer',
    'PLOAD_INC_ATM_KG': 'Atmospheric Deposition',
    'PLOAD_INC_URB_KM2': 'Urban Land',
    'PLOAD_INC_NFIX_M2': 'Red Alder Trees',
    'PLOAD_INC_STORAGE': 'Storage Lag',
    'PLOAD_INC_TOTAL_ND': 'Total without Aquatic Decay'
}
annual_load.rename(columns=rename_sources, inplace=True)
annual_mean_load.rename(columns=rename_sources, inplace=True)
quarterly_mean_load.rename(columns=rename_sources, inplace=True)

# 6) Export all three tables to a multi-sheet workbook
out1 = f'{SHARED}/annual_quarterly_loads.xlsx'
with pd.ExcelWriter(out1, engine='xlsxwriter') as writer:
    annual_load.to_excel(writer, sheet_name='Annual_Load', index=False)
    annual_mean_load.to_excel(writer, sheet_name='Annual_Mean_Load', index=False)
    quarterly_mean_load.to_excel(writer, sheet_name='Quarterly_Mean_Load', index=False)
print(f'✅ Written Puget TN loads → {out1}')

✅ Written Puget TN loads → /content/drive/MyDrive/SPARROW Google CoLab Files/annual_quarterly_loads.xlsx


In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# PART 2: Aggregate Wastewater Point-Source Inputs by WRIA
# ──────────────────────────────────────────────────────────────────────────────

# 1) Read in nutrient loads & facility attributes
df_loads = pd.read_csv(f'{SHARED}/nutrient_loads.csv', dtype={'FAC_ID': str})
df_attr  = pd.read_excel(
    f'{SHARED}/fac_attributes_detailed.xlsx',
    dtype={'FAC_ID': str}, engine='openpyxl'
)

# 2) Merge on FAC_ID
to_bring = ['FAC_ID','FAC_NAME','SSM_ID','FAC_TYPE','WRIA_NR','WRIA_NM','FAC_DISCHARGE']
merged = pd.merge(df_loads, df_attr[to_bring], on='FAC_ID', how='left')

# 3) Compute days_in_month safely (Feb = 28 days)
def safe_days(r):
    y, m = r['YEAR'], r['MONTH']
    if pd.isna(y) or pd.isna(m):
        return np.nan
    return 28 if int(m)==2 else calendar.monthrange(int(y),int(m))[1]
merged['days_in_month'] = merged.apply(safe_days, axis=1)

# 4) Coerce relevant columns to numeric
numeric_cols = ['FLOW_MGD','NH4N_MG_L','TKN_MG_L','NO2NO3N_MG_L','TN_MG_L','days_in_month']
for c in numeric_cols:
    merged[c] = pd.to_numeric(merged[c], errors='coerce')

# 5) Build TN concentration
merged['TN_CONC_MG_L'] = np.where(
    merged['TN_MG_L'].notna(),
    merged['TN_MG_L'],
    merged[['TKN_MG_L','NO2NO3N_MG_L']].fillna(0).sum(axis=1)
)

# 6) Calculate TN_LOAD_KG_MO_CALC
mask = merged[['FLOW_MGD','TN_CONC_MG_L','days_in_month']].notna().all(axis=1)
merged['TN_LOAD_KG_MO_CALC'] = np.nan
merged.loc[mask, 'TN_LOAD_KG_MO_CALC'] = (
    merged.loc[mask, 'FLOW_MGD'] *
    merged.loc[mask, 'TN_CONC_MG_L'] *
    3.786 *
    merged.loc[mask, 'days_in_month']
)

# 7) Override for industrial & fish hatchery
industrial = ['sic_INDU','sic_0921']
merged.loc[merged['FAC_TYPE'].isin(industrial), 'TN_LOAD_KG_MO_CALC'] = merged['TN_LOAD_KG_MO']

# 8) Rename & reorder columns
merged = merged.rename(columns={'TN_CONC_MG_L':'TN_CONC_MG_L_CALC'})
explicit = [
    'FAC_ID','YEAR','MONTH','TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC',
    'FLOW_MGD','TN_MG_L','TN_CONC_MG_L_CALC','days_in_month',
    'NH4N_MG_L','NO2NO3N_MG_L','TKN_MG_L',
    'FAC_NAME','SSM_ID','FAC_TYPE','WRIA_NR','WRIA_NM'
]
final_cols = explicit + [c for c in merged.columns if c not in explicit]
merged = merged[final_cols]

# 9) Combine WRIA 3 & 4
merged['WRIA_NR'] = merged['WRIA_NR'].astype(object)
num = pd.to_numeric(merged['WRIA_NR'], errors='coerce')
mask34 = num.isin([3,4])
merged.loc[mask34,'WRIA_NR']='3&4'
merged.loc[mask34,'WRIA_NM']='Skagit - Samish'

# 10) Build WRIA‑Year summary in Gg/year & rename
summary = (
    merged
    .groupby(['WRIA_NR','WRIA_NM','YEAR'])[['TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC']]
    .sum()
    .reset_index()
)
# convert kg → Gg
summary[['TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC']] /= 1e6
# rename to Gg units
summary.rename(columns={
    'TN_LOAD_KG_MO':      'TN_LOAD_GG_MO',
    'TN_LOAD_KG_MO_CALC': 'TN_LOAD_GG_MO_CALC'
}, inplace=True)

# 11) Compute mean annual load in Gg/year
annual_mean = (
    summary
    .groupby(['WRIA_NR','WRIA_NM'])[['TN_LOAD_GG_MO','TN_LOAD_GG_MO_CALC']]
    .mean()
    .reset_index()
)

# 12) Compute mean quarterly load in each WRIA
#   a) assign quarter labels
merged['quarter'] = merged['MONTH'].map({
    1:'Winter',2:'Winter',3:'Winter',
    4:'Spring',5:'Spring',6:'Spring',
    7:'Summer',8:'Summer',9:'Summer',
    10:'Fall',11:'Fall',12:'Fall'
})
#   b) sum per WRIA‑quarter‑year
quarterly_by_year = (
    merged
    .groupby(['WRIA_NR','WRIA_NM','quarter','YEAR'])[['TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC']]
    .sum()
    .reset_index()
)
# convert kg → Gg & rename
quarterly_by_year[['TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC']] /= 1e6
quarterly_by_year.rename(columns={
    'TN_LOAD_KG_MO':      'TN_LOAD_GG_MO',
    'TN_LOAD_KG_MO_CALC': 'TN_LOAD_GG_MO_CALC'
}, inplace=True)
#   d) seasonal mean
quarterly_mean = (
    quarterly_by_year
    .groupby(['WRIA_NR','WRIA_NM','quarter'])[['TN_LOAD_GG_MO','TN_LOAD_GG_MO_CALC']]
    .mean()
    .reset_index()
)

# 13) Export all four sheets to Excel and download
local_path = '/content/Wastewater_Inputs_WRIA.xlsx'
with pd.ExcelWriter(local_path, engine='xlsxwriter') as writer:
    merged.to_excel(writer, sheet_name='Merged', index=False)
    summary.to_excel(writer, sheet_name='WRIA_Year_Summary_Gg', index=False)
    annual_mean.to_excel(writer, sheet_name='Annual_Mean_Gg', index=False)
    quarterly_mean.to_excel(writer, sheet_name='Quarterly_Mean_Gg', index=False)
import os
from google.colab import files
files.download(local_path)

# 13b) Also save to Drive
drive_folder = '/content/drive/MyDrive/SPARROW Google CoLab Files'
os.makedirs(drive_folder, exist_ok=True)
drive_path = os.path.join(drive_folder,'Wastewater_Inputs_WRIA.xlsx')
with pd.ExcelWriter(drive_path, engine='xlsxwriter') as writer:
    merged.to_excel(writer, sheet_name='Merged', index=False)
    summary.to_excel(writer, sheet_name='WRIA_Year_Summary_Gg', index=False)
    annual_mean.to_excel(writer, sheet_name='Annual_Mean_Gg', index=False)
    quarterly_mean.to_excel(writer, sheet_name='Quarterly_Mean_Gg', index=False)

print(f"✅ Workbooks exported to:\n • Local: {local_path}\n • Drive: {drive_path}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Workbooks exported to:
 • Local: /content/Wastewater_Inputs_WRIA.xlsx
 • Drive: /content/drive/MyDrive/SPARROW Google CoLab Files/Wastewater_Inputs_WRIA.xlsx


In [ ]:
# %% [code]
# ————————————————————————————————————————————————————————————————
# SCENARIO: 3 mg/L Apr–Oct
# ————————————————————————————————————————————————————————————————
import numpy as np
import pandas as pd
from google.colab import drive

# 1) Override TN_MG_L to 3 mg/L for April–October
mask_mo = merged['MONTH'].between(4, 10)
merged.loc[mask_mo, 'TN_MG_L'] = 3

# 2) Recompute TN_CONC_MG_L_CALC
merged['TN_CONC_MG_L_CALC'] = np.where(
    merged['TN_MG_L'].notna(),
    merged['TN_MG_L'],
    merged[['TKN_MG_L','NO2NO3N_MG_L']].fillna(0).sum(axis=1)
)

# 3) Recompute TN_LOAD_KG_MO_CALC
mask_valid = merged[['FLOW_MGD','TN_CONC_MG_L_CALC','days_in_month']].notna().all(axis=1)
merged.loc[~mask_valid, 'TN_LOAD_KG_MO_CALC'] = np.nan
merged.loc[mask_valid, 'TN_LOAD_KG_MO_CALC'] = (
    merged.loc[mask_valid, 'FLOW_MGD']
  * merged.loc[mask_valid, 'TN_CONC_MG_L_CALC']
  * 3.786
  * merged.loc[mask_valid, 'days_in_month']
)

# 4) Re‑apply industrial/fish‑hatchery override
industrial = ['sic_INDU','sic_0921']
merged.loc[merged['FAC_TYPE'].isin(industrial), 'TN_LOAD_KG_MO_CALC'] = merged['TN_LOAD_KG_MO']

# 5) Build WRIA‑Year summary (kg → Gg + rename)
summary = (
    merged
    .groupby(['WRIA_NR','WRIA_NM','YEAR'])[['TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC']]
    .sum()
    .reset_index()
)
summary[['TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC']] /= 1e6
summary.rename(columns={
    'TN_LOAD_KG_MO':      'TN_LOAD_GG_MO',
    'TN_LOAD_KG_MO_CALC': 'TN_LOAD_GG_MO_CALC'
}, inplace=True)

# 6) Mean annual load in each WRIA (Gg/year)
annual_mean = (
    summary
    .groupby(['WRIA_NR','WRIA_NM'])[['TN_LOAD_GG_MO','TN_LOAD_GG_MO_CALC']]
    .mean()
    .reset_index()
)

# 7) Build Quarterly‑by‑Year (kg → Gg + rename), then seasonal mean
#    a) quarterly_by_year
quarterly_by_year = (
    merged
    .groupby(['WRIA_NR','WRIA_NM','quarter','YEAR'])[['TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC']]
    .sum()
    .reset_index()
)
quarterly_by_year[['TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC']] /= 1e6
quarterly_by_year.rename(columns={
    'TN_LOAD_KG_MO':      'TN_LOAD_GG_MO',
    'TN_LOAD_KG_MO_CALC': 'TN_LOAD_GG_MO_CALC'
}, inplace=True)

#    b) seasonal average
quarterly_mean = (
    quarterly_by_year
    .groupby(['WRIA_NR','WRIA_NM','quarter'])[['TN_LOAD_GG_MO','TN_LOAD_GG_MO_CALC']]
    .mean()
    .reset_index()
)

# 8) Compute differences
merged['TN_LOAD_DIFF_KG_MO']       = merged['TN_LOAD_KG_MO_CALC']    - merged['TN_LOAD_KG_MO']
summary['TN_LOAD_DIFF_GG_YEAR']    = summary['TN_LOAD_GG_MO_CALC']   - summary['TN_LOAD_GG_MO']
annual_mean['TN_LOAD_DIFF_GG_YEAR']= annual_mean['TN_LOAD_GG_MO_CALC']- annual_mean['TN_LOAD_GG_MO']
quarterly_mean['TN_LOAD_DIFF_GG']  = quarterly_mean['TN_LOAD_GG_MO_CALC'] - quarterly_mean['TN_LOAD_GG_MO']

# 9) Export to Drive
out_path = '/content/drive/My Drive/SPARROW Google CoLab Files/3 mg Seasonal Scenario.xlsx'
with pd.ExcelWriter(out_path, engine='xlsxwriter') as writer:
    merged.to_excel(writer, sheet_name='Merged (kg)', index=False)
    summary.to_excel(writer, sheet_name='WRIA_Year_Summary_Gg', index=False)
    annual_mean.to_excel(writer, sheet_name='Annual_Mean_Gg', index=False)
    quarterly_mean.to_excel(writer, sheet_name='Quarterly_Mean_Gg', index=False)

print(f"✅ Exported “3 mg Seasonal Scenario” workbook to:\n{out_path}")


✅ Exported “3 mg Seasonal Scenario” workbook to:
/content/drive/My Drive/SPARROW Google CoLab Files/3 mg Seasonal Scenario.xlsx


In [ ]:
# %% [code]
# ————————————————————————————————————————————————————————————————
# SCENARIO: BNR 8 (cool) /5 (warm) /3 (hot)
# ————————————————————————————————————————————————————————————————
import numpy as np
import pandas as pd
from google.colab import drive

# 1) Override TN_MG_L based on season:
# 8 mg/L Nov–Mar
mask_nov_mar = (merged['MONTH'] >= 11) | (merged['MONTH'] <= 3)
merged.loc[mask_nov_mar, 'TN_MG_L'] = 8

# 5 mg/L Apr–Jun and October
mask_apr_jun_oct = merged['MONTH'].between(4, 6) | (merged['MONTH'] == 10)
merged.loc[mask_apr_jun_oct, 'TN_MG_L'] = 5

# 3 mg/L Jul–Sep
mask_jul_sep = merged['MONTH'].between(7, 9)
merged.loc[mask_jul_sep, 'TN_MG_L'] = 3

# (Month 10 remains at whatever value it already had)


# 2) Recompute TN_CONC_MG_L_CALC
merged['TN_CONC_MG_L_CALC'] = np.where(
    merged['TN_MG_L'].notna(),
    merged['TN_MG_L'],
    merged[['TKN_MG_L','NO2NO3N_MG_L']].fillna(0).sum(axis=1)
)

# 3) Recompute TN_LOAD_KG_MO_CALC
mask_valid = merged[['FLOW_MGD','TN_CONC_MG_L_CALC','days_in_month']].notna().all(axis=1)
merged.loc[~mask_valid, 'TN_LOAD_KG_MO_CALC'] = np.nan
merged.loc[mask_valid, 'TN_LOAD_KG_MO_CALC'] = (
    merged.loc[mask_valid, 'FLOW_MGD']
  * merged.loc[mask_valid, 'TN_CONC_MG_L_CALC']
  * 3.786
  * merged.loc[mask_valid, 'days_in_month']
)

# 4) Re‑apply industrial/fish‑hatchery override
industrial = ['sic_INDU','sic_0921']
merged.loc[merged['FAC_TYPE'].isin(industrial), 'TN_LOAD_KG_MO_CALC'] = merged['TN_LOAD_KG_MO']

# 5) Build WRIA‑Year summary (kg → Gg + rename)
summary = (
    merged
    .groupby(['WRIA_NR','WRIA_NM','YEAR'])[['TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC']]
    .sum()
    .reset_index()
)
summary[['TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC']] /= 1e6
summary.rename(columns={
    'TN_LOAD_KG_MO':      'TN_LOAD_GG_MO',
    'TN_LOAD_KG_MO_CALC': 'TN_LOAD_GG_MO_CALC'
}, inplace=True)

# 6) Mean annual load in each WRIA (Gg/year)
annual_mean = (
    summary
    .groupby(['WRIA_NR','WRIA_NM'])[['TN_LOAD_GG_MO','TN_LOAD_GG_MO_CALC']]
    .mean()
    .reset_index()
)

# 7) Build Quarterly‑by‑Year (kg → Gg + rename), then seasonal mean
#    a) quarterly_by_year
quarterly_by_year = (
    merged
    .groupby(['WRIA_NR','WRIA_NM','quarter','YEAR'])[['TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC']]
    .sum()
    .reset_index()
)
quarterly_by_year[['TN_LOAD_KG_MO','TN_LOAD_KG_MO_CALC']] /= 1e6
quarterly_by_year.rename(columns={
    'TN_LOAD_KG_MO':      'TN_LOAD_GG_MO',
    'TN_LOAD_KG_MO_CALC': 'TN_LOAD_GG_MO_CALC'
}, inplace=True)

#    b) seasonal average
quarterly_mean = (
    quarterly_by_year
    .groupby(['WRIA_NR','WRIA_NM','quarter'])[['TN_LOAD_GG_MO','TN_LOAD_GG_MO_CALC']]
    .mean()
    .reset_index()
)

# 8) Compute differences
merged['TN_LOAD_DIFF_KG_MO']       = merged['TN_LOAD_KG_MO_CALC']    - merged['TN_LOAD_KG_MO']
summary['TN_LOAD_DIFF_GG_YEAR']    = summary['TN_LOAD_GG_MO_CALC']   - summary['TN_LOAD_GG_MO']
annual_mean['TN_LOAD_DIFF_GG_YEAR']= annual_mean['TN_LOAD_GG_MO_CALC']- annual_mean['TN_LOAD_GG_MO']
quarterly_mean['TN_LOAD_DIFF_GG']  = quarterly_mean['TN_LOAD_GG_MO_CALC'] - quarterly_mean['TN_LOAD_GG_MO']

# 9) Export to Drive
out_path = '/content/drive/My Drive/SPARROW Google CoLab Files/8.5.3 Scenario.xlsx'
with pd.ExcelWriter(out_path, engine='xlsxwriter') as writer:
    merged.to_excel(writer, sheet_name='Merged (kg)', index=False)
    summary.to_excel(writer, sheet_name='WRIA_Year_Summary_Gg', index=False)
    annual_mean.to_excel(writer, sheet_name='Annual_Mean_Gg', index=False)
    quarterly_mean.to_excel(writer, sheet_name='Quarterly_Mean_Gg', index=False)

print(f"✅ Exported “8.5.3 Scenario” workbook to:\n{out_path}")


✅ Exported “8.5.3 Scenario” workbook to:
/content/drive/My Drive/SPARROW Google CoLab Files/8.5.3 Scenario.xlsx
